In [ ]:
## Title: Prediction of House Prices using R

Introduction

1.....Load Package

2.....Read data

3.....Data Exploration

4.....Variable Selection  

5.....Missing Value

6.....Data Visualization

7.....Build the model

Import Libraries

In [ ]:
Lib


In [1]:
data <- read.csv(file = '../input/housing/data.csv')

head(data)




In [ ]:
tail(data)